# Retrain a SSD MobilNet model for object detection with Tensorflow1

This notebook allows you to train a network of neurons for object detection.
You will use Tensorflow 1 to train a quantified MobilNet SSD model (V1 or V2).
The interesting thing about training your model on Tensorflow 1 is that the models offered on *Tensorflow Model Zoo* are already quantified and therefore allow to use the network on TPU.

To use your trained network on TPU you can look at this notebook: link

This training is done on a GPU configuration : 


### File tree : 

tod_tf1  

├── pre_trained   
│	└── <pre_trained-network>   
├── training  
│   └──<project>  
│       ├── <pre_trained-network>  
│       ├── train.record  
│       ├── test.record  
│       ├── pipeline.config 
│       ├── form_detection.pbtxt
│       ├── images   
│           ├── test  
│           │   └── *.jpg, *.png ... *.xml  
│           ├── train  
│           │   └── *.jpg, *.png ... *.xml  
│           └── *.csv  
│
    
        
└── models  
    └── research  
        └── object_detection
    

## 1. Installation 

#### Create a virtual environnement TF1 : 

In [ ]:
! conda create -n tf1 python=3.6
! conda activate tf1 
! mkdir tod_tf1 
! cd tod_tf1/
#téléchargement du git 

#### Installation of Tensorflow on your environnement :

In [ ]:
! pip uninstall tensorflow -y
! pip install tensorflow-gpu==1.15
# pip install tensorflow==1.15 # for a CPU 

#### Installation des models Tensorflow et de l'API google coral : 

In [ ]:
! git clone https://github.com/tensorflow/models.git

In [ ]:
! cd models && git checkout f788046ca876a8820e05b0b48c1fc2e16b0955bc

In [ ]:
! cd ..
! git clone https://github.com/google-coral/tutorials.git
! cp -r tutorials/docker/object_detection/scripts/* models/research/

#### Installation of Cython : 

In [ ]:
sudo apt-get install -y python python-tk
pip install Cython contextlib2 pillow lxml jupyter matplotlib

#### Installation of protoc : 

In [ ]:
! wget https://www.github.com/google/protobuf/releases/download/v3.0.0/protoc-3.0.0-linux-x86_64.zip
! unzip protoc-3.0.0-linux-x86_64.zip -d proto3
! mkdir -p local/bin && mkdir -p local/include
! mv proto3/bin/* local/bin
! mv proto3/include/* local/include
! rm -rf proto3 protoc-3.0.0-linux-x86_64.zip

#### Installation of cocoapi : 

In [ ]:
! git clone --depth 1 https://github.com/cocodataset/cocoapi.git
! (cd cocoapi/PythonAPI && make -j8)
! cp -r cocoapi/PythonAPI/pycocotools/ models/research/
! rm -rf cocoapi

PYTHONPATH : 

## 2. Create your dataset 

To retrain a model you have to create your **own dataset** : 
1. Take picture of the object that you have to detect (if you work on the Reachy robot go to this notebook : like)
2. Annotate images with labelImg -> *.xml files
3. Convert XML annotated files to CSV format
4. Convert CSV annotated files to tensorflow record format
5. Create the file label_map.pbtxt which contains the labels of the objects to be recognized.

For all of this step go to https://learn.e.ros4.pro/fr/vision/object_detection_tf2/ section 5. 

## 3. Training

Download the model *ssd_mobilenet_v2_quantized_coco* on the tensorflow model zoo : https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf1_detection_zoo.md

In [ ]:
! cd models/research
! ./retrain_detection_model.sh --num_training_steps 50000 --num_eval_steps 100

## 4.  Export the network from the . pb format to the . tflite format:

In [ ]:
! export ckpt_number=50000
! python object_detection/export_tflite_ssd_graph.py   --pipeline_config_path="${CKPT_DIR}/pipeline.config"   --trained_checkpoint_prefix="${TRAIN_DIR}/model.ckpt-${ckpt_number}"   --output_directory="${OUTPUT_DIR}"   --add_postprocessing_op=true
! tflite_convert   --output_file="${OUTPUT_DIR}/output_tflite_graph.tflite"   --graph_def_file="${OUTPUT_DIR}/tflite_graph.pb"   --inference_type=QUANTIZED_UINT8   --input_arrays="${INPUT_TENSORS}"   --output_arrays="${OUTPUT_TENSORS}"   --mean_values=128   --std_dev_values=128   --input_shapes=1,300,300,3   --change_concat_input_ranges=false   --allow_nudging_weights_to_use_fast_gemm_kernel=true   --allow_custom_ops

## 5. Compile for edgeTPU

### 5.1 Dowload the edgeTPU compiler 

In [ ]:
! curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
! echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" | sudo tee /etc/apt/sources.list.d/coral-edgetpu.list
! sudo apt-get update
! sudo apt-get install edgetpu-compiler

### 5.2 Turn tflite into an edgeTPU compatible version:

In [ ]:
! cd ~/tod_tf1/training/form_detection/ssd_mobilenet_v1_quantized_300x300_coco_2019_01_03/model1/tflite
! edgetpu_compiler output_tflite_graph.tflite

## 6. Use your trained network on the TPU : 

I use my network on the Reachy robot with this notebook : link 